In [ ]:
import random 

# import dataframe from filtered_intersection_df_interpolated.xlsx

# Number of individuals in each generation 
POPULATION_SIZE = 1000

# Valid genes
# genes for each column in the dataframe based on its min and max values (range) 
GENES = '''abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOP 
QRSTUVWXYZ 1234567890, .-;:_!"#%&/()=?@${[]}'''

# find the standard deviation of each column in the dataframe
# pick one data from dataframe as current situation
# save total_coal and total_fuel to a variable
# these are the "output_columns": ['metal_temp', 'ni_met', 'c_met', 'si_met', 'fe_met', 's_met', 'ni_slag',
#       'fe_slag', 't_kalsin', 'pic_161', 'loi_kalsin']
# save the output_columns to a variable
# check these columns as "input_columns": ['ni_in', 'fe_in', 'cao_in', 'al2o3_in', 's_m', 'bc', 'mc_kilnfeed',
#       'fc_coal', 'gcv_coal', 'current_pry', 'current_sec1', 'current_sec2',
#       'current_sec3', 'load', 'power_factor', 'realisasi_beban', 'rpm',
#       'kg_tco', 'charge_kiln', 'tdo', 'pry_p', 'sec_p', 'pry_v', 'sec_v',
#       'total_coal', 'total_fuel', 'a_f_ratio', 'reductor_consume', 't_tic162',
#       't_tic163']
# these columns as "fixed _columns" is fixed: 'ni_in', 'fe_in', 'cao_in', 'al2o3_in', 's_m', 'bc', 'mc_kilnfeed', 'fc_coal', 'gcv_coal', 'kg_tco', 'charge_kiln', 'tdo'. The rest of input_columns needs to be optimized as "optimized_columns"

# chromosome is created from optimized_columns
# use xgboost_model_90_10.pkl to predict output_columns based on fixed_columns and optimized_columns
# calculate the fitness score based on the difference between predicted output_columns and actual output_columns
# try for 200 generations to find the optimized_columns


class Individual(object): 
	''' 
	Class representing individual in population 
	'''
	def __init__(self, chromosome): 
		self.chromosome = chromosome 
		self.fitness = self.cal_fitness() 

	@classmethod
	def mutated_genes(self): 
		''' 
		create random genes for mutation 
		'''
		global GENES 
		gene = random.choice(GENES) 
		return gene 

	@classmethod
	def create_gnome(self): 
		''' 
		create chromosome or string of genes 
		'''
		global TARGET 
		gnome_len = len(TARGET) 
		return [self.mutated_genes() for _ in range(gnome_len)] 

	def mate(self, par2): 
		''' 
		Perform mating and produce new offspring 
		'''

		# chromosome for offspring 
		child_chromosome = [] 
		for gp1, gp2 in zip(self.chromosome, par2.chromosome):	 

			# random probability 
			prob = random.random() 

			# if prob is less than 0.45, insert gene 
			# from parent 1 
			if prob < 0.45: 
				child_chromosome.append(gp1) 

			# if prob is between 0.45 and 0.90, insert 
			# gene from parent 2 
			elif prob < 0.90: 
				child_chromosome.append(gp2) 

			# otherwise insert random gene(mutate), 
			# for maintaining diversity 
			else: 
				child_chromosome.append(self.mutated_genes()) 

		# create new Individual(offspring) using 
		# generated chromosome for offspring 
		return Individual(child_chromosome) 

	def cal_fitness(self): 
		''' 
		Calculate fitness score, it is the number of 
		characters in string which differ from target 
		string. 
		'''
		global TARGET 
		fitness = 0
		for gs, gt in zip(self.chromosome, TARGET): 
			if gs != gt: fitness+= 1
		return fitness 

# Driver code 
def main(): 
	global POPULATION_SIZE 

	#current generation 
	generation = 1

	found = False
	population = [] 

	# create initial population 
	for _ in range(POPULATION_SIZE): 
				gnome = Individual.create_gnome() 
				population.append(Individual(gnome)) 

	while not found: 

		# sort the population in increasing order of fitness score 
		population = sorted(population, key = lambda x:x.fitness) 

		# if the individual having lowest fitness score ie. 
		# 0 then we know that we have reached to the target 
		# and break the loop 
		if population[0].fitness <= 0: 
			found = True
			break

		# Otherwise generate new offsprings for new generation 
		new_generation = [] 

		# Perform Elitism, that mean 10% of fittest population 
		# goes to the next generation 
		s = int((10*POPULATION_SIZE)/100) 
		new_generation.extend(population[:s]) 

		# From 50% of fittest population, Individuals 
		# will mate to produce offspring 
		s = int((90*POPULATION_SIZE)/100) 
		for _ in range(s): 
			parent1 = random.choice(population[:50]) 
			parent2 = random.choice(population[:50]) 
			child = parent1.mate(parent2) 
			new_generation.append(child) 

		population = new_generation 

		print("Generation: {}\tString: {}\tFitness: {}".format(generation, "".join(population[0].chromosome), population[0].fitness)) 

		generation += 1

	
	print("Generation: {}\tString: {}\tFitness: {}".format(generation, "".join(population[0].chromosome), population[0].fitness)) 

if __name__ == '__main__': 
	main() 


Generation: 1	String: d Yik? GII]{oGpo N}f	Fitness: 17
Generation: 2	String: jVl=i7  xC$7$ rGr0Pv	Fitness: 16
Generation: 3	String: IVl=iU Lkbt8- rGG0ks	Fitness: 13
Generation: 4	String: IVl=iU i1bkq-RrGe0ks	Fitness: 11
Generation: 5	String: I lAve L/O&su.rGe:kw	Fitness: 9
Generation: 6	String: Ivlove Ce,k$UarGeeks	Fitness: 6
Generation: 7	String: I 
ov7 Ge,kH-orGeeks	Fitness: 5
Generation: 8	String: I love GejkUforGe:ks	Fitness: 3
Generation: 9	String: I love Ge4ksforGeeks	Fitness: 1
Generation: 10	String: I love Ge4ksforGeeks	Fitness: 1
Generation: 11	String: I love GeeksforGeeks	Fitness: 0
